In [863]:
#!unzip "/content/drive/MyDrive/train_dataset_Самолет2.zip" -d "/content/"

In [864]:
!pip install catboost

In [865]:
import pandas as pd
import numpy as np
import random,os

In [866]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(0)

In [867]:
import pickle

In [868]:
best_models=[]

In [869]:
for i in range(3):
  with open(f'model{i}.pkl','rb') as f:
    best_models.append(pickle.load(f))

test_data=np.loadtxt('test_data.txt')

In [870]:
armatura=29
inventory=0
bal=0

In [871]:
test_prices=pd.read_excel('test.xlsx')[:-3]
len(test_prices)

25

In [872]:
len(test_data)

25

In [873]:
best_zakup=1174600

In [874]:
signal_cnt=random.randint(0,6)
signal=False
zakups=[]
enough=False
trend=False
for i,row in enumerate(test_data):
    trend=False
    if i>29-inventory-1:
      zakup=0
      enough=True
    else:
      zakup=1
    queue=[row[0]]
    for model in best_models:
        queue.append(model.predict(row))
    down_m=np.mean([queue[1],queue[0]])
    up_m=np.mean([queue[3],queue[2]])

    if up_m>down_m:
        signal=True
        print('signal')
    if signal:
        signal_cnt-=1

    if signal_cnt==0:
        if np.mean([queue[1]-queue[0],queue[2]-queue[1],queue[3]-queue[2]])>0 and not enough:
            zakup=10-inventory
            print('trend change')
        signal=False
        signal_cnt=6


    bal-=zakup*test_prices['Цена на арматуру'].values[i]
    inventory+=zakup

    zakups.append(zakup)
    zakup=1
    inventory-=1
    print(down_m,up_m,np.mean([queue[1]-queue[0],queue[2]-queue[1],queue[3]-queue[2]]),inventory,zakup,test_prices['Цена на арматуру'].values[i],enough,signal_cnt)
print(bal,inventory)
print(bal/best_zakup)

signal
0.4606517857375576 0.4945646827285331 -0.003232871305337596 0 1 46000 False 5
0.4189539970814359 0.4051996214326815 -0.034152962065324144 0 1 46000 False 4
0.38768808227262985 0.3708284818527695 -0.030831772508830218 0 1 46000 False 3
0.37008754762441715 0.32125233190641633 -0.04207386311666019 0 1 45500 False 2
signal
0.3670503272159205 0.3699030604079161 -0.022059669743024295 0 1 45500 False 1
signal
0.349763545889592 0.38365509815778653 -0.007647670282079466 0 1 45000 False 6
signal
0.30187271038950825 0.40643512637559287 0.02899012486039772 0 1 44000 False 5
signal
0.2901494285574112 0.4375144013729092 0.06224077588491452 0 1 43000 False 4
signal
0.26008032910773654 0.38359454106493507 0.08256668236401296 0 1 42000 False 3
signal
0.1651543141525899 0.2554060162075408 0.05201288363812345 0 1 41500 False 2
signal
0.10956409586746454 0.287631805728243 0.08253640817955409 0 1 41000 False 1
signal
trend change
0.06464702992664191 0.2587735212233868 0.0910850270429215 9 1 40000 Fa

In [876]:
zakups=zakups+[0,0,0]

In [878]:
sub=pd.read_excel('test.xlsx')
sub['Объем']=zakups

In [879]:
sub.to_excel('submission.xlsx', index=False)

In [880]:
sub

,dt,Цена на арматуру,Объем
0,2022-09-05,46000,1
1,2022-09-12,46000,1
2,2022-09-19,46000,1
3,2022-09-26,45500,1
4,2022-10-03,45500,1
5,2022-10-10,45000,1
6,2022-10-17,44000,1
7,2022-10-24,43000,1
8,2022-10-31,42000,1
9,2022-11-07,41500,1
